In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report

# Define ImageDataGenerators for training and validation with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=[0.2, 1.5]
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    r"C:\Users\Priyamvadha Pradeep\Desktop\FYP\CNN\New Models\train",
    target_size=(224, 224),  # MobileNetV2 input size
    batch_size=32,
    class_mode='binary'  # Binary classification: ALS and Healthy
)

val_generator = val_datagen.flow_from_directory(
    r"C:\Users\Priyamvadha Pradeep\Desktop\FYP\CNN\New Models\validate",
    target_size=(224, 224),  # MobileNetV2 input size
    batch_size=32,
    class_mode='binary'
)

# Load pre-trained MobileNetV2 model with ImageNet weights (not including the top layers)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model initially
for layer in base_model.layers:
    layer.trainable = False

# Create the model by adding custom layers on top of the base model
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())  # For dimensionality reduction
model.add(Dropout(0.5))  # Add dropout to reduce overfitting
model.add(Dense(256, activation='relu'))  # Dense layer
model.add(Dropout(0.5))  # Dropout
model.add(Dense(1, activation='sigmoid'))  # Binary classification output

# Compile the model with a lower learning rate to start
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)

# Train the model with the base model frozen
history = model.fit(
    train_generator,
    epochs=10,  # Fine-tuning after 10 epochs can be adjusted
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Unfreeze the last few layers of the base model for fine-tuning
for layer in base_model.layers[-30:]:  # Unfreeze the last 30 layers (you can adjust this number)
    layer.trainable = True

# Re-compile the model to apply the fine-tuning
model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with fine-tuned layers
fine_tune_history = model.fit(
    train_generator,
    epochs=30,  # Continue training for more epochs
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Get predictions for confusion matrix
y_pred = model.predict(val_generator)
y_pred = (y_pred > 0.5)  # Convert probabilities to class labels

# True labels
y_true = val_generator.classes

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
report = classification_report(y_true, y_pred, target_names=train_generator.class_indices.keys())
print("Classification Report:")
print(report)

Found 978 images belonging to 2 classes.
Found 246 images belonging to 2 classes.
Epoch 1/10


c:\Users\Priyamvadha Pradeep\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/31 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.5133 - loss: 0.9432 - val_accuracy: 0.6667 - val_loss: 0.6304 - learning_rate: 1.0000e-04
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.5987 - loss: 0.7945 - val_accuracy: 0.6667 - val_loss: 0.6039 - learning_rate: 1.0000e-04
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.6117 - loss: 0.7503 - val_accuracy: 0.6667 - val_loss: 0.6029 - learning_rate: 1.0000e-04
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.6279 - loss: 0.6811 - val_accuracy: 0.6789 - val_loss: 0.6094 - learning_rate: 1.0000e-04
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.5894 - loss: 0.7583 - val_accuracy: 0.6748 - val_loss: 0.6049 - learning_rate: 1.0000e-04
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.5901 - loss: 0.7294 - val_accuracy: 0.6667 - val_loss: 0.6009 - learning_rate: 1.0000e-04
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.5742 - loss: 0.7521 - v